In [38]:
import pandas as pd
import numpy as np
import cobra
import re

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import ContinuousScoreIntegrationStrategy
from troppo.methods.reconstruction.tINIT import tINIT, tINITProperties

patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)
# load model and expression data
model = cobra.io.read_sbml_model('/home/biodata/aman/Human-GEM/model/Human-GEM.xml')
expression_data = pd.read_csv('/home/biodata/aman/expression_data_ensembl.csv', index_col=0)
#print(expression_data.head())

expression_data_transposed = expression_data.T

#print("\nTransposed shape:", expression_data_transposed.shape)
#print(expression_data_transposed.head())

omics_container = TabularReader(path_or_df=expression_data_transposed, 
                                nomenclature='gene',
                                omics_type='transcriptomics').to_containers()

single_sample = omics_container[0]

model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts)

data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader,
                                                 and_func=min, or_func=sum)

def score_apply(reaction_map_scores):
    return {k:0  if v is None else v for k, v in reaction_map_scores.items()}

continuous_integration = ContinuousScoreIntegrationStrategy(score_apply=score_apply)
scores = continuous_integration.integrate(data_map=data_map)
print(f'\nScores are:\n {scores.items()}')
# check
# print(next(iter(single_sample.data.keys())))
# print(model.genes[0].id)
essential_reactions = [model_wrapper.model_reader.r_ids.index(rid) for rid in ['MAR13082']] # biomass doesn't make sense. change to smtg else perhaps # user input here

#['MAR04413', 'MAR09931', 'MAR09932', 'MAR12140','MAR10024', 'MAR10062', 'MAR10063', 'MAR10064','MAR10065', 'MAR13082']

# Create the properties for the tINIT algorithm.
properties = tINITProperties(reactions_scores=[v for k, v in scores.items()], solver='CPLEX', essential_reactions=essential_reactions )

# Run the tINIT algorithm.
tinit = tINIT(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

model_tinit = tinit.run()
# model.reactions
# len(model.reactions)

selected_reactions = [model.reactions[i] for i in model_tinit.flatten().tolist()]
print(f'length of the retained/selected reactions: {len(selected_reactions)}') # versus total reactions also # print(len(model.reactions), len(ctx_model.reactions))
ctx_model = model.copy()

selected_ids = [r.id for r in selected_reactions]
to_remove = [r for r in ctx_model.reactions if r.id not in selected_ids]

ctx_model.remove_reactions(to_remove, remove_orphans=True)

print(ctx_model)
# export
cobra.io.write_sbml_model(ctx_model, "tinit_context_specific_model.xml")
print('model exported')

solution = ctx_model.optimize()
print(solution.objective_value)

#get the model summary
ctx_model.summary()


Scores are:
 dict_items([('MAR03905', 78.286562788), ('MAR03907', 44.24426), ('MAR04097', 31.771797), ('MAR04099', 98.04613599999999), ('MAR04108', 31.771797), ('MAR04133', 31.771797), ('MAR04281', 506.59981), ('MAR04388', 507.765747424), ('MAR04283', 5.73763936), ('MAR08357', 225.6702612), ('MAR04379', 277.10958), ('MAR04301', 277.10958), ('MAR04355', 188.00741567), ('MAR04358', 353.61657154), ('MAR04360', 0), ('MAR04363', 674.694227), ('MAR04365', 196.491128), ('MAR04368', 154.47135642), ('MAR04370', 134.68111499999998), ('MAR04371', 196.491128), ('MAR04372', 134.68111499999998), ('MAR04373', 1215.1982752000001), ('MAR04375', 802.45645567), ('MAR04377', 0.85911185), ('MAR04381', 136.00047), ('MAR04391', 305.41425), ('MAR04394', 113.80163007), ('MAR04396', 30.728589), ('MAR04521', 29.82345269), ('MAR06412', 20.081738), ('MAR07745', 3.804638), ('MAR07747', 39.304367), ('MAR08360', 46.637497), ('MAR08652', 29.82345269), ('MAR08757', 11.996882), ('MAR03989', 108.3554822), ('MAR04122', 1

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                1
CPXPARAM_QPMethod                                1
CPXPARAM_Simplex_Tolerances_Optimality           1e-08
CPXPARAM_Simplex_Tolerances_Feasibility          1e-08
CPXPARAM_MIP_Tolerances_MIPGap                   0.001
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-09
Tried aggregator 3 times.
MIP Presolve eliminated 1290 rows and 2311 columns.
MIP Presolve modified 22 coefficients.
Aggregator did 1117 substitutions.
Reduced MIP has 24993 rows, 61699 columns, and 129920 nonzeros.
Reduced MIP has 18512 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.14 sec. (162.31 ticks)
Probing fixed 37 vars, tightened 295 bounds.
Probing time = 0.08 sec. (12.23 ticks)
Tried aggregator 2 times.
Detecting symmetries...
MIP Presolve eliminated 287 rows and 767 columns.
Aggregator did 2 substitutions.
Reduced MIP h

In [19]:
#tinit.sol.__dict__['_Solution__value_map']

AttributeError: 'tINIT' object has no attribute 'sol'